### NORMALIZE CREATING NEW TABLE

ta metoda polega na tworzeniu tabeli dla każdej listy

In [2]:
import pandas as pd
import json
import pyarrow as pa
from tqdm import tqdm

In [4]:
paths_test = [ '../data-test/airlines.json', 
            '../data-test/gists.json', 
            '../data-test/historical-events.json',#have to rename the same value in json (movie on  the same level)
            '../data-test/movies.json',
            '../data-test/reddit.json',#bad
            '../data-test/nasa.json'
            ]

In [5]:
paths = [ '../data/airlines.json', 
            '../data/gists.json', 
            '../data/historical-events.json',
            '../data/movies.json',
            '../data/reddit.json',
            '../data/nasa.json']

In [6]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                attribute_name= name + a +'.'
                flatten(x[a], attribute_name)
        elif type(x) is list:
            i = 0
            for a in x:
                attribute_name=name.rstrip('.') + '[' + str(i) + '].'
                flatten(a, attribute_name)
                i += 1
        else:
            out[name.rstrip('.')] = x

    flatten(y)
    return out

results = []

for path in paths_test:
    with open(path, encoding='utf-8') as f:
        dt = json.load(f)
        
    if isinstance(dt, list):
        flattened_data = [flatten_json(item) for item in dt]
    else:
        flattened_data = [flatten_json(dt)]

    df = pd.DataFrame(flattened_data)

    table = pa.Table.from_pandas(df)
    results.append(table)


In [ ]:
import pandas as pd
import pyarrow as pa
import json

def make_new_json(lists,table_name):
    print(table_name)
    data = []
    for i in range(len(lists)):
        for item in lists[i]:
            item_data={
                "row_number": i,
                "value": item
            }
            data.append(item_data)
    return data

def delete_lists(df,name):
    
    print("#"*24)
    print("#"*((24-len(name))//2)+name+"#"*((24-len(name))//2))
    print("#"*24)

    df=pd.json_normalize(df)

    is_list = df.apply(lambda x: any(isinstance(i, list) for i in x))

    list_columns = is_list[is_list].index.tolist()
    if(len(list_columns)>0):
        list_df = df[list_columns].copy()

        df = df.drop(list_columns, axis=1)
        df['row_number'] = range(1, len(df) + 1)
    main_table = pa.Table.from_pandas(df)
    print(main_table)
    if(len(list_columns)>0):
        for dropped_list in list_columns:
            new_json=make_new_json(list_df[dropped_list],dropped_list)
            delete_lists(new_json,dropped_list)

with open(path, encoding='utf-8') as f:
    dt = json.load(f)

delete_lists(dt,'main')

